# Compare 2017 Child Tax Credit to repeal

Investigates why tax units would be worse off when using the 2017 CTC than a repealed CTC, as Sean Wang discovered when running my CTC -> child benefit notebook using the PUF ([notebook](https://github.com/GoFroggyRun/Notebook/blob/master/ctc_ubi_puf.ipynb)).

Findings:
* 21k CPS records have a higher after-tax income when repealing the CTC as compared to with the 2017 CTC. 85k records have the reverse, as expected.
* 60% of these records have `n24 == 0`, which should be unaffected by any CTC changes. No records with `n24 == 0` had a lower after-tax income after repealing the CTC vs. 2017 CTC.
* The unexpected records have higher after expanded income than those with expected behavior.

*This behavior is still present when removing `"_DependentCredit_before_CTC": [False]` from the 2017 reform.*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np

In [2]:
tc.__version__

'0.19.0'

### Settings

In [3]:
pd.set_option('precision', 2)

### Utilities

In [4]:
def weighted_sum(df, col):
    return (df[col] * 1.0 * df.s006).sum()

def weighted_mean(df, col):
    return weighted_sum(df, col) / df.s006.sum()

### Create reforms

CTC repeal involves eliminating the normal CTC as well as the new dependent credit for children.

A reform to return to 2017 CTC law is used to show how the distribution of CTC benefits changed with TCJA.

In [5]:
noctc_reform = {
    2018: {
        '_CTC_c': [0],
        '_DependentCredit_Child_c': [0]
    }
}

y2017_reform = {
    2018: {
        # Current: 1400.0
        "_CTC_c": [1000.0],
        # Current: [200000.0, 400000.0, 200000.0, 200000.0, 400000.0]
        "_CTC_ps": [[75000.0, 110000.0, 55000.0, 75000.0, 75000.0]],
        # Current: 2500.0
        "_ACTC_Income_thd": [3000.0],
        # Current: 600.0
        "_DependentCredit_Child_c": [0.0],
        # Current: True
        "_DependentCredit_before_CTC": [False]
    }
}

## Generate data

In [6]:
recs = tc.Records.cps_constructor()

In [7]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [8]:
def static_calc(ctc_treatment='keep',
                year=2018,
                cols=['s006', 'aftertax_income', 'expanded_income',
                      'n24', 'XTOT']):
    """Creates static Calculator.

    Args:
        ctc_treatment: How the Child Tax Credit is treated. Options include:
            * 'keep': No change. Default.
            * 'repeal': End entirely.
            * 'y2017': Use 2017 law.
        year: Year to advance calculations to.
        cols: Columns to extract per Calculator record. 
            Defaults to ['s006', 'expanded_income', 'aftertax_income', 'nu18',
            'n24', 'XTOT'].
        
    Returns:
        DataFrame with `cols` and percentile, decile, and quintile of 
        after-tax income.
    """
    pol = tc.Policy()
    # Enact reform based on ctc_treatment.
    # Repeal CTC unless it's kept.
    if ctc_treatment == 'y2017':
        pol.implement_reform(y2017_reform)
    elif ctc_treatment == 'repeal':
        pol.implement_reform(noctc_reform)
    # Calculate. This is needed to calculate the revenue-neutral UBI.
    calc = tc.Calculator(records=recs, policy=pol, verbose=False)
    calc.advance_to_year(year)
    calc.calc_all()
    # Create DataFrame and add identifiers.
    df = calc.dataframe(cols)
    # Add weighted sums.
    df['s006_m'] = df.s006 / 1e6
    df['expanded_income_m'] = df.s006_m * df.expanded_income
    # Add identifier.
    df['ctc_treatment'] = ctc_treatment
    # What's the column for the ID?
    df['id'] = df.index
    return df

In [9]:
scenarios = pd.concat([
    static_calc(ctc_treatment='keep'),
    static_calc(ctc_treatment='y2017'),
    static_calc(ctc_treatment='repeal')])

## Preprocess

In [10]:
tu = scenarios.pivot_table(values='aftertax_income',
                           index='id', columns='ctc_treatment').reset_index()
tu.columns = ['id', 'afti_keep', 'afti_repeal', 'afti_y2017']

In [11]:
# Dimensions based on tax unit and baseline.
base_aftiq = scenarios.loc[scenarios.ctc_treatment == 
                           'keep'].drop('ctc_treatment', axis=1)

In [12]:
tu = pd.merge(tu, base_aftiq, on='id')

In [13]:
tu['y2017_vs_repeal'] = np.where(tu.afti_y2017 > tu.afti_repeal, 'Higher',
                                 np.where(tu.afti_y2017 < tu.afti_repeal,
                                          'Lower', 'Same'))
tu['has_n24'] = tu.n24 > 0
tu['records'] = 1

In [14]:
tu_pos = tu[tu.y2017_vs_repeal == 'Higher']
tu_neg = tu[tu.y2017_vs_repeal == 'Lower']

## Analysis

In [15]:
tu.pivot_table(index='y2017_vs_repeal', values=['records', 's006_m'],
               aggfunc=sum)

,records,s006_m
y2017_vs_repeal,,
Higher,85518,33.95
Lower,20731,4.01
Same,350216,131.93


In [16]:
x = tu.pivot_table(index='y2017_vs_repeal', columns='has_n24',
                   values=['records', 's006_m', 'expanded_income_m'], aggfunc=sum)
# There's a better way to do this using slices.
x['mean_expanded_income_false'] = x.expanded_income_m[0] / x.s006_m[0]
x['mean_expanded_income_true'] = x.expanded_income_m[1] / x.s006_m[1]
x

expanded_income_m             records           s006_m         \
has_n24                     False      True     False     True   False   True   
y2017_vs_repeal                                                                 
Higher                        NaN  2.23e+06       NaN  85518.0     NaN  33.95   
Lower                    5.01e+05  2.61e+05   12685.0   8046.0    2.81   1.20   
Same                     9.23e+06  1.66e+06  295117.0  55099.0  121.22  10.71   

                mean_expanded_income_false mean_expanded_income_true  
has_n24                                                               
y2017_vs_repeal                                                       
Higher                                 NaN                  65767.77  
Lower                            178236.16                 217137.93  
Same                              76168.83                 155145.58

In [17]:
print(('The average people per tax unit among negative tax units is {:0.2f} '
       'vs. {:1.2f} among positive tax units.').format(
    weighted_mean(tu_neg, 'XTOT'), weighted_mean(tu_pos, 'XTOT')))

The average people per tax unit among negative tax units is 3.42 vs. 3.21 among positive tax units.


### Tax units with higher aftertax_income without CTC than with 2017 CTC

In [18]:
tu_neg[tu_neg.n24 == 0].sample(2)

,id,afti_keep,afti_repeal,afti_y2017,s006,aftertax_income,expanded_income,n24,XTOT,s006_m,expanded_income_m,y2017_vs_repeal,has_n24,records
20796,20796,67569.95,67569.95,67224.69,728.71,67569.95,88133.33,0.0,2.0,7.29e-04,64.22,Lower,False,1
273231,273231,254454.75,254454.75,253954.75,46.78,254454.75,325671.56,0.0,3.0,4.68e-05,15.23,Lower,False,1


In [19]:
tu_neg[tu_neg.n24 > 0].sample(2)

,id,afti_keep,afti_repeal,afti_y2017,s006,aftertax_income,expanded_income,n24,XTOT,s006_m,expanded_income_m,y2017_vs_repeal,has_n24,records
214572,214572,207269.92,205269.92,204769.92,37.17,207269.92,264340.92,1.0,4.0,3.72e-05,9.83,Lower,True,1
444597,444597,224126.55,222126.55,221126.55,33.46,224126.55,256418.90,1.0,5.0,3.35e-05,8.58,Lower,True,1


### No tax units have higher aftertax_income without CTC than with 2018 CTC

In [20]:
print(('{:0.0f} tax units have higher after-tax income without CTC than '
       'with 2018 CTC').format(tu[tu.afti_repeal > tu.afti_keep].shape[0]))

0 tax units have higher after-tax income without CTC than with 2018 CTC


## Create tax records

Start with a record with unexpected result, then re-test after simplifying the record (zeroing out various inputs) until a minimal case is identified.

Without a way to convert Records to dataframes and back, or to index Records, unclear how to do this.

In [21]:
recs[455159]

TypeError: 'Records' object does not support indexing